In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch import nn, optim
from tensorflow import keras
from torch.utils.data import Dataset
import torch.nn.functional as F

/home/cyy/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-05 10:45:24.946115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 10:45:24.946742: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
class MNISTDataset(Dataset):
    def __init__(self, image, label):
        #self.image = image.clone().detach # torch.tensor(image, dtype = torch.float32)
        self.image  = image.to(torch.float32)
        self.label = label

    def __len__(self): return len(self.label)

    def __getitem__(self, idx):
        image_ = self.image[idx]
        image_ = image_[None, :]
        label_ = self.label[idx]
        return image_, label_
    

        

In [5]:
(x_tr, y_tr), (x_ts, y_ts) = keras.datasets.mnist.load_data()

In [14]:


# selecting only label 0 to 5
x, y = [], []
for i in range(len(x_tr)):
    if y_tr[i] < 5:
        x.append(x_tr[i])
        y.append(y_tr[i])
x_tr, y_tr = torch.from_numpy(np.stack(x)), torch.from_numpy(np.stack(y))
x, y = [], []
for i in range(len(x_ts)):
    if y_ts[i] < 5:
        x.append(x_ts[i])
        y.append(y_ts[i])
x_ts, y_ts = torch.from_numpy(np.stack(x)), torch.from_numpy(np.stack(y))
print(x_tr.shape)

x_tr, y_tr = x_tr[0:6400], y_tr[0:6400]
x_ts, y_ts = x_ts[0:640], y_ts[0:640]
trainset = MNISTDataset(x_tr, y_tr)
valset = MNISTDataset(x_ts, y_ts)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([6400, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [15]:
torch.manual_seed(75)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 5)   # change according to the num of classes

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [16]:
n_epochs = 1    # change the epoch
learning_rate = 0.006
momentum = 0.5
log_interval = 1  # change for output

networks, opts = [], []

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)


train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(trainloader.dataset) for i in range(n_epochs + 1)]


In [17]:
def rademacher(shape, device='cpu'):
    """Creates a random tensor of size [shape] under the Rademacher distribution (P(x=1) == P(x=-1) == 0.5)"""
    x = torch.empty(shape, device=device)
    x.random_(0, 2)  # Creates random tensor of 0s and 1s
    x[x == 0] = -1  # Turn the 0s into -1s
    return x

def listify(x):
    """If x is already a list, do nothing. Otherwise, wrap x in a list."""
    if isinstance(x, list):
        return x
    else:
        return [x]

def multi_layer_second_directional_derivative(G, z, x, G_z, epsilon, **G_kwargs):
    """Estimates the second directional derivative of G w.r.t. its input at z in the direction x"""
    G_to_x = G(z + x, **G_kwargs)
    G_from_x = G(z - x, **G_kwargs)

    G_to_x = listify(G_to_x)
    G_from_x = listify(G_from_x)
    G_z = listify(G_z)

    eps_sqr = epsilon ** 2
    sdd = [(G2x - 2 * G_z_base + Gfx) / eps_sqr for G2x, G_z_base, Gfx in zip(G_to_x, G_z, G_from_x)]
    return sdd

def stack_var_and_reduce(list_of_activations, reduction=torch.max):
    """Equation (5) from the paper."""
    second_orders = torch.stack(list_of_activations)  # (k, N, C, H, W)
    var_tensor = torch.var(second_orders, dim=0, unbiased=True)  # (N, C, H, W)
    penalty = reduction(var_tensor)  # (1,) (scalar)
    return penalty


def multi_stack_var_and_reduce(sdds, reduction=torch.max, return_separately=False):
    """Iterate over all activations to be regularized, then apply Equation (5) to each."""
    sum_of_penalties = 0 if not return_separately else []
    for activ_n in zip(*sdds):
        penalty = stack_var_and_reduce(activ_n, reduction)
        sum_of_penalties += penalty if not return_separately else [penalty]
    return sum_of_penalties

def hessian_penalty(G, z, k=2, epsilon=0.1, reduction=torch.max, return_separately=False, G_z=None, **G_kwargs):
    if G_z is None:
        G_z = G(z, **G_kwargs)
    rademacher_size = torch.Size((k, *z.size()))  # (k, N, z.size())
    xs = epsilon * rademacher(rademacher_size, device=z.device)
    second_orders = []
    for x in xs:  # Iterate over each (N, z.size()) tensor in xs
        central_second_order = multi_layer_second_directional_derivative(G, z, x, G_z, epsilon, **G_kwargs)
        second_orders.append(central_second_order)  # Appends a tensor with shape equal to G(z).size()
    loss = multi_stack_var_and_reduce(second_orders, reduction, return_separately)  # (k, G(z).size()) --> scalar
    return loss

In [80]:
# derivative = [[]]*10

def train(epoch, para = 10):
  
  i = 0
  network.train()
  for batch_idx, (data, target) in enumerate(trainloader):
    data.requires_grad_(True)
    
    optimizer.zero_grad()
    output = network(data)
    # print(data.shape)  # [[64, 1, 28, 28]]
    # print(output.shape) # [64, 10]
    
    second_derivative_loss = 0
    for r in range(len(data)): # iterate all batches
          for j in range(5):  # change for num of labels
                if j != y_tr[i]:
                      def f(sample):
                            res = network(sample)
                        #     print(res.shape) [1, 10]
                            res = res.view(5)  # change for num of labels
                            return res[j]
                      H_matrix = torch.autograd.functional.hessian(f, data[r])
                  #     print(H_matrix.shape)  [1, 28, 28, 1, 28, 28]
                      H_matrix = H_matrix.view(28, 28, 28, 28)
                      trace = 0
                      for a in range(28):
                            for b in range(28):
                                  trace += abs(H_matrix[a][b][a][b])  #abs
                      second_derivative_loss += trace
                
    
    loss = F.nll_loss(output, target)
    loss.backward(retain_graph=True)
    

    second_derivative_loss = para*second_derivative_loss
    loss += second_derivative_loss
    loss.backward(retain_graph=True)

    optimizer.step()
    if batch_idx % log_interval == 0:
      print("derivative loss:", second_derivative_loss)
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(trainloader.dataset),
        100. * batch_idx / len(trainloader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
         (batch_idx*64) + ((epoch-1)*len(trainloader.dataset)))
    
    # derivative = new_derivative
    # new_derivative = []*10

In [81]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in valloader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(valloader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valloader.dataset),
    100. * correct / len(valloader.dataset)))

In [82]:
# # test()
# for epoch in range(1, n_epochs + 1):
#   train(epoch)
#   test()

In [83]:
with open('features_UMAP.npy', 'rb') as f:
    train_x = np.load(f)
    train_y_label = np.load(f)
    test_x = np.load(f)
    test_y = np.load(f)
    
train_x, train_y_label, test_x, test_y = \
    torch.from_numpy(train_x), torch.from_numpy(train_y_label), \
        torch.from_numpy(test_x), torch.from_numpy(test_y)


In [84]:
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel

In [85]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # self.covar_module = gpytorch.kernels.ScaleKernel(RBFKernel())
        self.covar_module = gpytorch.kernels.MaternKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [86]:
def scores(loader):
    network.eval()
    # outputs = [] 
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            output = network(data)
            # outputs.append(output)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))
    return output

In [87]:
n = 2000
all_index = np.arange(0, n)
models, likelihoods, mlls, opts = [], [], [], []
lr = 0.1

def model_initialize_GP():


    for j in range(5): # change for num of labels
        mo, l, ml, o = [], [], [], []
        for k in range(5):  # change for num of labels
            val_index = np.arange(np.floor(n/5)*k, np.floor(n/5)*(k+1))  # change for num of labels
            train_index = [i for i in all_index if i not in val_index]
            x = train_x[train_index, :]
            y = log_y[train_index, j]
            
            likelihood = gpytorch.likelihoods.GaussianLikelihood()
            model = ExactGPModel(x, y, likelihood)
            likelihood.train()
            model.train()
            mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
            
            l.append(likelihood)
            mo.append(model)
            ml.append(mll)
            
            optimizer = torch.optim.Adam(model.parameters(), lr = lr)  # Includes GaussianLikelihood parameters
            o.append(optimizer)
            
        models.append(mo)
        likelihoods.append(l)
        mlls.append(ml)
        opts.append(o)
        
    # models[0][1] represents label 0 and the first fold
        

In [88]:
pred_train_y = []
def train_GP():
    training_iter = 10
    # train_y = torch.load('train_score.pt') : x axis

    for j in range(5):  # j represents the 10 labels # change for num of labels
        p = []
        for k in range(5): # k represents the k fold cross validation  # change for num of labels
            print("Model", j, ", Cross Validation Group", k)
            # initialize the model
            val_index = np.arange(np.floor(n/5)*k, np.floor(n/5)*(k+1))  # change for num of labels
            train_index = [i for i in all_index if i not in val_index]
            tr_x = train_x[train_index, :]
            val_x = train_x[val_index, :]
            tr_y = log_y[train_index, j]
            # val_y = train_y[val_index, k]
            likelihood = likelihoods[j][k]
            model = models[j][k]
            mll = mlls[j][k]
            optimizer = opts[j][k]
            
            # training
            for i in range(training_iter):                
                # Zero gradients from previous iteration
                optimizer.zero_grad()
                # Output from model
                output = model(tr_x)
                # print(output.shape, tr_y.shape)
                
                # Calc loss and backprop gradients
                loss = -mll(output, tr_y)
                loss.backward()
                
                if i % 4 == 0:
                    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                        i + 1, training_iter, loss.item(),
                        #model.covar_module.base_kernel.lengthscale.item(),
                        0.1,
                        model.likelihood.noise.item()
                    ))
                optimizer.step()
            
                # evaluation of the current model
            model.eval()
            likelihood.eval()
            with gpytorch.settings.fast_pred_var(), torch.no_grad():
                test_dist = model(val_x)
                pred_means = test_dist.loc
                p.append(pred_means)
        p = torch.stack(p)
        pred_train_y.append(p)        
    

In [89]:
test()
parameters = [0.1, 1, 10, 100]
for para in parameters:
    
    
    network = Net()
    optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
    for epoch in range(1, n_epochs + 1):
        train(epoch, para)
        test()
        
    trloader = torch.utils.data.DataLoader(trainset, batch_size=6400, shuffle=False)
    tsloader = torch.utils.data.DataLoader(valset, batch_size=640, shuffle=False)
    train_score = scores(trloader)
    # test_score = scores(tsloader)
    filename = "data/para" + str(para) + '_train_score.pt'
    torch.save(train_score, filename)
    # torch.save(test_score, 'test_score.pt')
    
    train_y = torch.load(filename)
    
    train_x, train_y = train_x[0: n], train_y[0: n, :]
    test_y = test_y.to(torch.int64)
    add = - torch.min(train_y) + 1
    log_y = add + train_y
    log_y = torch.log(log_y)
    
    
    models, likelihoods, mlls, opts = [], [], [], []
    model_initialize_GP()
    pred_train_y = []
    train_GP()
    
    
    for j in range(5):   # change for num of labels
        pred_train_y[j] = pred_train_y[j].view(1, -1).flatten()
        pred_train_y[j] = torch.exp(pred_train_y[j]) - add
        
    for j in range(5):     # change for num of labels
        name = "plots/para" + str(para) + "scaled_model" + str(j) +".png"
        x_data = train_y[:, j]
        mini = torch.min(x_data)
        if mini < -20:
            mini = -40
        else: 
            mini = -20
        y_data= pred_train_y[j]
        plt.plot(x_data, y_data, 'o', color='black')
        x = np.linspace(-10,2,100)
        y = x
        plt.plot(x, y, '-r', label='y=2x+1')
        plt.title('Model' + str(j))
        plt.xlabel('DNN training scores', color='#1C2833')
        plt.ylabel('GP training scores', color='#1C2833')
        plt.savefig(name, facecolor = 'white', transparent = False)
        plt.show()

/tmp/ipykernel_1772/161431047.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 6.7439, Accuracy: 27/640 (4%)

derivative loss: tensor(-0.0697)
Train Epoch: 1 [0/6400 (0%)]	Loss: 35.723858
derivative loss: tensor(-0.0216)
Train Epoch: 1 [64/6400 (1%)]	Loss: 29.912062
derivative loss: tensor(-1.2460)
Train Epoch: 1 [128/6400 (2%)]	Loss: 1.219437
derivative loss: tensor(-3.0843)
Train Epoch: 1 [192/6400 (3%)]	Loss: -1.065788
derivative loss: tensor(-2.3277)
Train Epoch: 1 [256/6400 (4%)]	Loss: -0.169231
derivative loss: tensor(-2.4885)
Train Epoch: 1 [320/6400 (5%)]	Loss: -0.421961
derivative loss: tensor(-2.6261)
Train Epoch: 1 [384/6400 (6%)]	Loss: -0.552436
derivative loss: tensor(-1.6370)
Train Epoch: 1 [448/6400 (7%)]	Loss: 0.275743
derivative loss: tensor(-2.2441)
Train Epoch: 1 [512/6400 (8%)]	Loss: -0.404477
derivative loss: tensor(-1.8840)
Train Epoch: 1 [576/6400 (9%)]	Loss: -0.027307
derivative loss: tensor(-1.7580)
Train Epoch: 1 [640/6400 (10%)]	Loss: 0.182201
derivative loss: tensor(-1.7440)
Train Epoch: 1 [704/6400 (11%)]	Loss: 0

KeyboardInterrupt: 

In [ ]:
# plt.scatter(y_tr[0:500], train_score[0:500, 0])
# plt.show()
# torch.scatter(train_score[0:200, 0], index = torch.from_numpy(y_tr[0:200]))